## Topic Modeling of Lyrics with LDA

Author: Miles Mezaki

In [14]:
import pandas as pd
import numpy as np

In [15]:
df = pd.read_csv('data/labeled_lyrics_cleaned.csv')
df

,Unnamed: 0,artist,seq,song,label
0,0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday,0.626
1,1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die,0.630
2,2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...,The Otherside,0.240
3,3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot,0.536
4,4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds,0.371
...,...,...,...,...,...
158348,158348,Adam Green,"And we live on borrowed time,\r\nBut this head...",Friends of Mine,0.737
158349,158349,Adam Green,Frozin in time forever\r\nCarrying that torch ...,Frozen in Time,0.482
158350,158350,Adam Green,Hard to be a girl. \r\nSo nice to be a boy. \r...,Hard to Be a Girl,0.733
158351,158351,Adam Green,"I want to chose to die,\r\nAnd be buried with ...",I Wanna Die,0.361


In [16]:
add_on = pd.read_csv('songs_with_lyrics.csv')
add_on['song'] = add_on['track_name']
add_on['seq'] = add_on['lyrics']
add_on

,file_name,track_name,artist,lyrics,song,seq
0,share_video_7329274198096973099_.mp3,Montagem Mysterious Game,LXNGVX,This song is an instrumental,Montagem Mysterious Game,This song is an instrumental
1,share_video_7339685917759769899_.mp3,Dumptruck,Kinfolk Thugs,[Intro]\nAh-where they at?\n(K-K-K-Kin Folk Th...,Dumptruck,[Intro]\nAh-where they at?\n(K-K-K-Kin Folk Th...
2,share_video_7324083003284573482_.mp3,Little Life,Cordelia,[Verse 1]\nHow would you have me described?\nW...,Little Life,[Verse 1]\nHow would you have me described?\nW...
3,share_video_7303373812387679490_.mp3,A Sky Full of Stars (Live at the Royal Albert ...,Coldplay,"[Intro]\nOh, turn the lights on, turn the ligh...",A Sky Full of Stars (Live at the Royal Albert ...,"[Intro]\nOh, turn the lights on, turn the ligh..."
4,share_video_7306351228974288159_.mp3,How To Never Stop Being Sad,dandelion hands,Repeat to yourself that they're not really gon...,How To Never Stop Being Sad,Repeat to yourself that they're not really gon...
...,...,...,...,...,...,...
363,NaN,In The Party,Flo Milli,La la la la la la Flo Milli shit La la la la l...,In The Party,La la la la la la Flo Milli shit La la la la l...
364,NaN,Beef FloMix,Flo Milli,Hah Flo Milli shit bitch I like cash and my ...,Beef FloMix,Hah Flo Milli shit bitch I like cash and my ...
365,NaN,Rodeo (Remix),Flo Milli,Yeah I wanna fuck you right now Reverse that...,Rodeo (Remix),Yeah I wanna fuck you right now Reverse that...
366,NaN,Conceited,Flo Milli,Thats Fyre Feelin myself Im conceited Huh Fe...,Conceited,Thats Fyre Feelin myself Im conceited Huh Fe...


In [17]:
add_on['artist'].isna().sum()

0

In [18]:
# merged_df = pd.merge(df, add_on, on=['artist','seq','song'], how='left')
# merged_df
print(len(df['artist']))
merged_df = pd.concat([df[['artist','seq','song']], add_on[['artist','seq','song']]], ignore_index=True, axis=0)
merged_df

158353


,artist,seq,song
0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday
1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die
2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...,The Otherside
3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot
4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds
...,...,...,...
158716,Flo Milli,La la la la la la Flo Milli shit La la la la l...,In The Party
158717,Flo Milli,Hah Flo Milli shit bitch I like cash and my ...,Beef FloMix
158718,Flo Milli,Yeah I wanna fuck you right now Reverse that...,Rodeo (Remix)
158719,Flo Milli,Thats Fyre Feelin myself Im conceited Huh Fe...,Conceited


In [19]:
merged_df.loc[merged_df['artist'] == "Flo Milli"]

,artist,seq,song
158457,Flo Milli,[Intro: Flo Milli]\nNever had a bitch like me ...,Never Lose Me (Mixed)
158551,Flo Milli,[Intro: Flo Milli]\nNever had a bitch like me ...,Never Lose Me (Mixed)
158630,Flo Milli,[Intro: Flo Milli]\nNever had a bitch like me ...,Never Lose Me (Mixed)
158659,Flo Milli,[Intro: Flo Milli]\nNever had a bitch like me ...,Never Lose Me (Mixed)
158715,Flo Milli,Oohlala Bangladesh I aint had no good dick i...,Edible (feat. Gunna)
158716,Flo Milli,La la la la la la Flo Milli shit La la la la l...,In The Party
158717,Flo Milli,Hah Flo Milli shit bitch I like cash and my ...,Beef FloMix
158718,Flo Milli,Yeah I wanna fuck you right now Reverse that...,Rodeo (Remix)
158719,Flo Milli,Thats Fyre Feelin myself Im conceited Huh Fe...,Conceited
158720,Flo Milli,Never had a bitch like me in your life You ain...,Never Lose Me


In [20]:

seen = set()
to_drop = []
for i,row in df.iterrows():
    if (row['artist'], row['song']) in seen:
        to_drop.append(i)
    seen.add((row['artist'], row['song']))

In [21]:
print(len(merged_df))
df = merged_df.drop(to_drop)
print(len(df))

158721
158680


In [22]:
to_drop

[9970,
 9971,
 9976,
 42862,
 43183,
 45717,
 45718,
 45719,
 50648,
 57111,
 57119,
 57150,
 74966,
 74967,
 75636,
 78347,
 83229,
 94212,
 96443,
 96445,
 96446,
 96447,
 108268,
 120201,
 125247,
 125277,
 125278,
 125280,
 125281,
 125282,
 125287,
 125288,
 125289,
 125290,
 125291,
 125292,
 125296,
 133164,
 141742,
 147526,
 155274]

In [23]:
df.iloc[9965:9975]

,artist,seq,song
9965,Andy Williams,"Moon river, wider than a mile\r\nI'm crossing ...","Moon River (From ""Breakfast at Tiffany's"")"
9966,Andy Williams,Night and you and blue Hawaii\r\nThe night is ...,Blue Hawaii
9967,Andy Williams,I found my thrill on blueberry hill\r\nOn blue...,Blueberry Hill
9968,Andy Williams,"You tell me you love me, you say you'll be tru...",Butterfly
9969,Andy Williams,He's got the whole world in his hands he's got...,He's Got the Whole World in His Hands
9972,Andy Williams,On a day like today\r\nWe pass the time away\r...,Love Letters in the Sand
9973,Andy Williams,This is the night of love\r\nThe shining hour ...,Love Song of Kalua
9974,Andy Williams,Evening shadows make me blue\r\nWhen each wear...,My Happiness
9975,Andy Williams,There's just one place for me near you\r\nIt's...,Near You
9977,Andy Williams,"Some enchanted evening, you may see a stranger...",Some Enchanted Evening


In [24]:
df = df.reset_index(drop=True)
df.iloc[9965:9975]

,artist,seq,song
9965,Andy Williams,"Moon river, wider than a mile\r\nI'm crossing ...","Moon River (From ""Breakfast at Tiffany's"")"
9966,Andy Williams,Night and you and blue Hawaii\r\nThe night is ...,Blue Hawaii
9967,Andy Williams,I found my thrill on blueberry hill\r\nOn blue...,Blueberry Hill
9968,Andy Williams,"You tell me you love me, you say you'll be tru...",Butterfly
9969,Andy Williams,He's got the whole world in his hands he's got...,He's Got the Whole World in His Hands
9970,Andy Williams,On a day like today\r\nWe pass the time away\r...,Love Letters in the Sand
9971,Andy Williams,This is the night of love\r\nThe shining hour ...,Love Song of Kalua
9972,Andy Williams,Evening shadows make me blue\r\nWhen each wear...,My Happiness
9973,Andy Williams,There's just one place for me near you\r\nIt's...,Near You
9974,Andy Williams,"Some enchanted evening, you may see a stranger...",Some Enchanted Evening


Define code to expand contracted words, which are prominent in music:

In [25]:
def expand_contractions(s):
    res = ""
    contractions = { 
        "ain't": "aint", # Could be many tenses, but is not is probably the best mapping for all tenses
        "aren't": "arent",
        "can't": "cant",
        "can't've": "cannot have",
        "'cause": "because",
        "could've": "couldve",
        "couldn't": "couldnt",
        "couldn't've": "couldnt have",
        "didn't": "didnt",
        "doesn't": "doesnt",
        "don't": "dont",
        "hadn't": "hadnt",
        "hadn't've": "had not have",
        "hasn't": "hasnt",
        "haven't": "havent",
        "he'd": "hed",
        "he'd've": "he wouldve",
        "he'll": "he will",
        "he'll've": "he will have",
        "he's": "hes",
        "how'd": "howd",
        "how'd'y": "howd you",
        "how'll": "how will",
        "how's": "hows",
        "I'd": "Id",
        "I'd've": "I wouldve",
        "I'll": "I will",
        "I'll've": "I will have",
        "I'm": "Im",
        "I've": "Ive",
        "isn't": "isnt",
        "it'd": "it would",
        "it'd've": "it wouldve",
        "it'll": "it will",
        "it'll've": "it will have",
        "it's": "its",
        "let's": "lets",
        "ma'am": "madam",
        "mayn't": "may not",
        "might've": "mightve",
        "mightn't": "might not",
        "mightn't've": "might not have",
        "must've": "mustve",
        "musta" : "mustve",
        "mustn't": "must not",
        "mustn't've": "must not have",
        "needn't": "need not",
        "needn't've": "need not have",
        "oughtn't": "ought not",
        "oughtn't've": "ought not have",
        "shan't": "shall not",
        "sha'n't": "shall not",
        "shan't've": "shall not have",
        "she'd": "shed",
        "she'd've": "she would have",
        "she'll": "shell",
        "she'll've": "she will have",
        "she's": "shes",
        "should've": "shouldve",
        "shouldn't": "shouldnt",
        "shouldn't've": "shouldnt have",
        "so've": "sove",
        "so's": "sos",
        "that'd": "thatd",
        "that'd've": "thatd have",
        "that's": "thats",
        "there'd": "thered",
        "there'd've": "thered have",
        "there's": "theres",
        "they'd": "theyd",
        "they'd've": "theyd have",
        "they'll": "theyll",
        "they'll've": "theyll have",
        "they're": "theyre",
        "they've": "theyve",
        "to've": "to have",
        "wasn't": "wasnt",
        "we'd": "wed",
        "we'd've": "we wouldve",
        "we'll": "we will",
        "we'll've": "we will have",
        "we're": "we are",
        "we've": "weve",
        "weren't": "werent",
        "what'll": "whatll",
        "what'll've": "whatll have",
        "what're": "whatre",
        "what's": "whats",
        "what've": "whatve",
        "when's": "whens",
        "when've": "whenve",
        "where'd": "whered",
        "where's": "wheres",
        "where've": "whereve",
        "who'll": "wholl",
        "who'll've": "wholl have",
        "who's": "whos",
        "who've": "whove",
        "why's": "whys",
        "why've": "whyve",
        "will've": "will have",
        "won't": "wont",
        "won't've": "wont have",
        "would've": "wouldve",
        "wouldn't": "wouldnt",
        "wouldn't've": "wouldnt have",
        "y'all": "yall",
        "y'all'd": "yall would",
        "y'all'd've": "yall would have",
        "y'all're": "yall are",
        "y'all've": "yall have",
        "you'd": "youd",
        "you'd've": "youd have",
        "you'll": "youll",
        "you'll've": "youll have",
        "you're": "youre",
        "you've": "youve"
        }
    for word in s.split():
        if word in contractions:
            res += contractions[word] + " "
        else:
            res += word + " "
    return res

print(df['seq'][0])

No, no
I ain't ever trapped out the bando
But oh Lord, don't get me wrong
I know a couple niggas that do
I'm from a place where everybody knows your name
They say I gotta watch my attitude
When they see money, man they all start actin' strange
So fuck with the ones that fuck with you
They can never say I'm brand new

It's everyday, everyday
Everyday, everyday, everyday
Everyday, everyday
Everyday, everyday
I've been talkin' my shit, nigga that's regular
I've been lovin' 'em thick, life is spectacular
I spend like I'ma die rich, nigga I'm flexin', yeah
Everyday, that's everyday
That's everyday
That's everyday
That's everyday, everyday

I see all of these wanna-be hot R&B singers
I swear you all sound the same
They start from the bottom, so far from the motto
You niggas'll never be Drake
Shout out to OVO
Most of them prolly don't know me though
I stay in the cut, I don't fuck with no
Body but I D, that's a pun on No I.D
When nobody know my name
Runnin' for my dream wasn't hard to do
You 

In [26]:
expand_contractions(df['seq'][0])

"No, no I aint ever trapped out the bando But oh Lord, dont get me wrong I know a couple niggas that do Im from a place where everybody knows your name They say I gotta watch my attitude When they see money, man they all start actin' strange So fuck with the ones that fuck with you They can never say Im brand new It's everyday, everyday Everyday, everyday, everyday Everyday, everyday Everyday, everyday Ive been talkin' my shit, nigga thats regular Ive been lovin' 'em thick, life is spectacular I spend like I'ma die rich, nigga Im flexin', yeah Everyday, thats everyday That's everyday That's everyday That's everyday, everyday I see all of these wanna-be hot R&B singers I swear you all sound the same They start from the bottom, so far from the motto You niggas'll never be Drake Shout out to OVO Most of them prolly dont know me though I stay in the cut, I dont fuck with no Body but I D, thats a pun on No I.D When nobody know my name Runnin' for my dream wasnt hard to do You break bread, I

Next define code that will match expressions missing the trailing g in their progressive tense and replace the apostrophe with a g. (e.g. trippin' -> tripping)

In [27]:
import re

pattern = r"(.)in'"
replacement =  r"\1ing"
def expand_verbs(s: str, pattern, replacement):
    """
    Assume s is the lyrics string
    """
    res, _ = re.subn(pattern, replacement, s)
    return res

expand_verbs("trippin', grippin', dribblin'.", pattern, replacement)

'tripping, gripping, dribbling.'

In [28]:
def expand_verbs_and_contractions(s, pattern=r"(.)in'", replacement=r"\1ing"):
    """
    s: string representing lyrics
    pattern: regular expression representing the pattern to be replaced
    replacement: regular expression to replace pattern

    Returns:
        res: string ready for tokenization
    """
    return expand_verbs(expand_contractions(s.lower()), pattern, replacement)
    
expand_verbs_and_contractions(df['seq'][0])

"no, no i aint ever trapped out the bando but oh lord, dont get me wrong i know a couple niggas that do i'm from a place where everybody knows your name they say i gotta watch my attitude when they see money, man they all start acting strange so fuck with the ones that fuck with you they can never say i'm brand new its everyday, everyday everyday, everyday, everyday everyday, everyday everyday, everyday i've been talking my shit, nigga thats regular i've been loving 'em thick, life is spectacular i spend like i'ma die rich, nigga i'm flexing, yeah everyday, thats everyday thats everyday thats everyday thats everyday, everyday i see all of these wanna-be hot r&b singers i swear you all sound the same they start from the bottom, so far from the motto you niggas'll never be drake shout out to ovo most of them prolly dont know me though i stay in the cut, i dont fuck with no body but i d, thats a pun on no i.d when nobody know my name running for my dream wasnt hard to do you break bread, 

In [29]:
df['seq'] = df['seq'].apply(expand_verbs_and_contractions)

In [30]:
df.head()

,artist,seq,song
0,Elijah Blake,"no, no i aint ever trapped out the bando but o...",Everyday
1,Elijah Blake,"the drinks go down and smoke goes up, i feel m...",Live Till We Die
2,Elijah Blake,she dont live on planet earth no more she foun...,The Otherside
3,Elijah Blake,"tripping off that grigio, mobbing, lights low ...",Pinot
4,Elijah Blake,"i see a midnight panther, so gallant and so br...",Shadows & Diamonds


Next we add code to remove the stop words in documents in a corpus.

In [31]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stop_words(sentence):
    word_tokens = word_tokenize(sentence)
    # converts the words in word_tokens to lower case and then checks whether 
    #they are present in stop_words or not
    filtered_sentence = [w.lower() for w in word_tokens if not w.lower() in stop_words]
    return " ".join(filtered_sentence)

remove_stop_words("this is an example Sentence Is it?")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'example sentence ?'

In [32]:
df['seq'] = df['seq'].apply(remove_stop_words)

Next, we want to tokenize and stem the data. We will not remove stop words since the people are generally important in lyrics and the verbs are often stopwords.

In [33]:
# from nltk.tokenize import word_tokenize 

# text = df['seq'][0]
# # tokenize
# result = word_tokenize(text)
# result

In [34]:
# from nltk.stem import PorterStemmer
 
# ps = PorterStemmer()
 
# # choose some words to be stemmed
# words = df['seq'][3]
 
# for w in words.split():
#     print(w, " : ", ps.stem(w))

In [35]:
# def token_stem(s):
#     """
#     Given a string s, tokenize the string, then stem it.
#     """
#     ps = PorterStemmer()
#     word_tokenize(s)
#     return " ".join([ps.stem(w) for w in word_tokenize(s)])

# token_stem("converts the words in word_tokens to lower case and then checks whether ")

Now we apply the tokenization and stemming to our lyrics.

In [36]:
# df['seq'] = df['seq'].apply(token_stem)

Now our lyrics text is ready for post-processing.

In [37]:
df['seq']

0         , aint ever trapped bando oh lord , dont get w...
1         drinks go smoke goes , feel , got let go cares...
2         dont live planet earth found love venus , that...
3         tripping grigio , mobbing , lights low trippin...
4         see midnight panther , gallant brave found , f...
                                ...                        
158675    la la la la la la flo milli shit la la la la l...
158676    hah flo milli shit bitch like cash hair ass da...
158677    yeah wan na fuck right reverse cowgirl im buck...
158678    thats fyre feelin im conceited huh feelin im c...
158679    never bitch like life aint never bitch like li...
Name: seq, Length: 158680, dtype: object

In [38]:
from sklearn.feature_extraction.text import CountVectorizer

In [39]:
# Initialize the vectorizer
vectorizer = CountVectorizer(
    strip_accents='unicode',
    stop_words='english',
    lowercase=True,
    token_pattern=r'\b[a-zA-Z]{3,}\b', # we want only words that contain letters and are 3 or more characters long
)

# Transform our data into the document-term matrix
dtm = vectorizer.fit_transform(df['seq'])
dtm

<158680x132809 sparse matrix of type '<class 'numpy.int64'>'
	with 8324630 stored elements in Compressed Sparse Row format>

In [40]:
feature_names = vectorizer.get_feature_names_out()
feature_names

array(['aaa', 'aaaa', 'aaaaa', ..., 'zzzz', 'zzzzs', 'zzzzzombieee'],
      dtype=object)

In [41]:
feature_names.shape

(132809,)

In [42]:
feature_names[300:350]

array(['abducted', 'abductee', 'abductees', 'abducting', 'abduction',
       'abductor', 'abducts', 'abdul', 'abdullah', 'abe', 'abeam',
       'abebe', 'abecedario', 'abed', 'abednago', 'abednego', 'abeds',
       'abeg', 'abeille', 'abeja', 'abejita', 'abel', 'abelard',
       'abelene', 'abeline', 'abell', 'abend', 'abende', 'abendigo',
       'abends', 'abenteuer', 'aber', 'abercrombie', 'aberdeen',
       'abernathy', 'aberrant', 'aberration', 'aberrations', 'aberrettes',
       'aberta', 'abetter', 'abetting', 'abeyance', 'abgefahren',
       'abgefickt', 'abgehn', 'abgesaugt', 'abgestellt', 'abgrund',
       'abgvll'], dtype=object)

In [43]:
doc1 = dtm[0]
doc1

<1x132809 sparse matrix of type '<class 'numpy.int64'>'
	with 77 stored elements in Compressed Sparse Row format>

In [44]:
row_index = 0
doc_vec = dtm.getrow(row_index).toarray()

non_zero_indices = doc_vec.nonzero()[1]
dtm_scores = doc_vec[0, non_zero_indices] # goes and retrieves the values corresponding to the non_zero_indices
words = [feature_names[i] for i in non_zero_indices]

for word, score in zip(words, dtm_scores):
    print(f"{word}: {score}")

acting: 2
aint: 2
attitude: 1
bando: 1
body: 1
brand: 1
bread: 1
break: 1
change: 1
changed: 1
couple: 1
crew: 1
cut: 1
die: 3
dont: 3
drake: 1
dream: 1
dress: 1
eat: 1
everybody: 1
everyday: 45
far: 1
flexing: 3
fuck: 3
funny: 1
gets: 1
got: 3
hard: 1
heard: 1
hobby: 1
hot: 1
know: 3
knows: 1
life: 3
like: 3
lonely: 1
lord: 1
lose: 1
loving: 4
man: 1
money: 2
motto: 1
new: 1
nigga: 6
niggas: 2
ones: 2
ovo: 1
place: 1
plate: 1
prolly: 1
pull: 1
pun: 1
regular: 3
rich: 3
running: 1
say: 2
shit: 3
shout: 1
singers: 1
sound: 1
spectacular: 3
spend: 3
start: 2
starved: 1
stay: 1
strange: 1
swear: 2
talking: 3
thats: 17
trapped: 1
wanna: 1
wasnt: 1
watch: 1
winning: 1
wrong: 1
yeah: 3
youre: 1


In [45]:
non_zero_indices

array([  1074,   2341,   6727,   8465,  12865,  14145,  14302,  14321,
        19111,  19114,  25249,  25902,  27079,  31001,  33169,  33806,
        33920,  34017,  35231,  38684,  38690,  40251,  42385,  44534,
        44865,  46455,  48143,  50919,  51709,  53436,  54364,  63328,
        63373,  66444,  66601,  67655,  67885,  67937,  68158,  69826,
        74913,  75767,  78448,  78780,  78800,  81465,  83167,  87568,
        87750,  90694,  91482,  91567,  94897,  96804,  98897, 100497,
       104150, 104521, 105447, 108520, 108982, 109074, 110479, 110511,
       110613, 111524, 113901, 114810, 116683, 119728, 127444, 127686,
       127761, 129677, 130722, 131293, 131908])

In [46]:
dtm.getcol(2327).toarray().T # get the column, turn it into an array format, then transpose it to be a row

array([[0, 0, 0, ..., 0, 0, 0]])

In [47]:
dtm.getcol(44592).toarray().T # Fuck, which should appear often in songs

array([[0, 0, 0, ..., 0, 0, 0]])

In [48]:
np.count_nonzero(dtm.getcol(44592).toarray().T)

342

Back to DF

In [49]:
def matrix2Doc(dtMatrix, features, index):
    """Turns each row of the document-term matrix into a list of terms"""
    row = dtMatrix.getrow(index).toarray()
    non_zero_indices = row.nonzero()[1]
    words = [features[idx] for idx in non_zero_indices]
    return words

In [50]:
allDocsAsTerms = [matrix2Doc(dtm, feature_names, i) for i in range(dtm.shape[0])]

In [51]:
len(allDocsAsTerms)

158680

In [52]:
df['terms'] = allDocsAsTerms
df.head()

,artist,seq,song,terms
0,Elijah Blake,", aint ever trapped bando oh lord , dont get w...",Everyday,"[acting, aint, attitude, bando, body, brand, b..."
1,Elijah Blake,"drinks go smoke goes , feel , got let go cares...",Live Till We Die,"[ace, aint, away, band, bite, blow, cares, cha..."
2,Elijah Blake,"dont live planet earth found love venus , that...",The Otherside,"[bad, bags, broke, broken, called, calling, ca..."
3,Elijah Blake,"tripping grigio , mobbing , lights low trippin...",Pinot,"[aint, baby, beginnings, blow, boy, calling, c..."
4,Elijah Blake,"see midnight panther , gallant brave found , f...",Shadows & Diamonds,"[answers, believe, brave, broke, coal, coffee,..."


In [53]:
from sklearn.decomposition import LatentDirichletAllocation

# Step 1: Initialize the model

lda = LatentDirichletAllocation(n_components=5, # based on GridSearch, we discover that 5 is a good number of topics for LDA.
                                random_state=0)

# Step 2: Fit the model
lda.fit(dtm)

LatentDirichletAllocation(n_components=5, random_state=0)

In [54]:
lda.components_

array([[ 0.20272087,  1.43276909,  0.2000002 , ...,  0.2006538 ,
         0.20133629,  0.20171972],
       [23.02984964,  5.94984246,  0.20167045, ...,  1.19934419,
         1.19846653,  1.18978735],
       [13.868423  ,  0.20082595,  4.19814044, ...,  0.20000084,
         0.2001966 ,  0.20000136],
       [10.26189862,  0.21167323,  0.20000053, ...,  0.20000077,
         0.20000038,  0.20692489],
       [ 0.63710786,  0.20488926,  0.20018838, ...,  0.2000004 ,
         0.20000019,  0.20156667]])

In [55]:
lda.components_.shape

(5, 132809)

In [56]:
def display_topics(model, features, no_top_words):
    """Helper function to show the top words of a model"""
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        print(" ".join([features[i]
                        for i in topic.argsort()[:-no_top_words-1:-1]])) # syntax for reversing a list [::-1]

display_topics(lda, feature_names, 20)

Topic 0:
know time dont away youre way like life day let got theres come say long gon feel night home right
Topic 1:
got like aint dont yeah know man gon nigga shit money make thats fuck rock want come right big cause
Topic 2:
love god sing come lord night life heart light heaven sweet dream song day world let eyes soul new jesus
Topic 3:
like man world black blood dead hes old war red white river water eyes die said hell people death theres
Topic 4:
love dont baby know want youre got yeah like girl say need make heart little feel come tell wan let


In [65]:
doc_topic_dist = lda.transform(dtm)
doc_topic_dist 

array([[1.90746712e-01, 6.66680149e-01, 1.10349550e-03, 1.09717424e-03,
        1.40372470e-01],
       [7.51059004e-01, 2.46266710e-01, 8.92413082e-04, 8.90735712e-04,
        8.91136634e-04],
       [6.63613769e-01, 1.30625843e-01, 1.66952957e-03, 1.65464586e-03,
        2.02436212e-01],
       ...,
       [5.08565023e-04, 6.95660221e-01, 5.01742665e-04, 3.82945557e-02,
        2.65034916e-01],
       [9.23030357e-04, 9.96298048e-01, 9.23446576e-04, 9.24289011e-04,
        9.31186091e-04],
       [1.09001566e-03, 5.93911033e-01, 1.07223745e-03, 1.07186636e-03,
        4.02854848e-01]])

In [66]:
doc_topic_dist.shape

(158680, 5)

In [67]:
def displayHeader(model, features, no_top_words):
    """Helper function to show the top words of a model"""
    topicNames = []
    for topic_idx, topic in enumerate(model.components_):
        topicNames.append(f"Topic {topic_idx}: " + (", ".join([features[i]
                             for i in topic.argsort()[:-no_top_words-1:-1]])))
    return topicNames

In [68]:
# column names
topicnames = displayHeader(lda, feature_names, 10)

# index names
docnames = df.index.tolist() # We will use the original names of the documents

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(doc_topic_dist, 3), 
                                 columns=topicnames, 
                                 index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1) # finds the maximum argument
df_document_topic['dominant_topic'] = dominant_topic

df_document_topic.head()

,"Topic 0: know, time, dont, away, youre, way, like, life, day, let","Topic 1: got, like, aint, dont, yeah, know, man, gon, nigga, shit","Topic 2: love, god, sing, come, lord, night, life, heart, light, heaven","Topic 3: like, man, world, black, blood, dead, hes, old, war, red","Topic 4: love, dont, baby, know, want, youre, got, yeah, like, girl",dominant_topic
0,0.191,0.667,0.001,0.001,0.140,1
1,0.751,0.246,0.001,0.001,0.001,0
2,0.664,0.131,0.002,0.002,0.202,0
3,0.199,0.690,0.001,0.001,0.109,1
4,0.125,0.322,0.168,0.003,0.382,4


In [69]:
df_document_topic.iloc[9970]

Topic 0: know, time, dont, away, youre, way, like, life, day, let          0.749
Topic 1: got, like, aint, dont, yeah, know, man, gon, nigga, shit          0.005
Topic 2: love, god, sing, come, lord, night, life, heart, light, heaven    0.005
Topic 3: like, man, world, black, blood, dead, hes, old, war, red          0.005
Topic 4: love, dont, baby, know, want, youre, got, yeah, like, girl        0.235
dominant_topic                                                             0.000
Name: 9970, dtype: float64

In [70]:
df_document_topic['song'] = df['song']
df_document_topic['artist'] = df['artist']
df_document_topic

,"Topic 0: know, time, dont, away, youre, way, like, life, day, let","Topic 1: got, like, aint, dont, yeah, know, man, gon, nigga, shit","Topic 2: love, god, sing, come, lord, night, life, heart, light, heaven","Topic 3: like, man, world, black, blood, dead, hes, old, war, red","Topic 4: love, dont, baby, know, want, youre, got, yeah, like, girl",dominant_topic,song,artist
0,0.191,0.667,0.001,0.001,0.140,1,Everyday,Elijah Blake
1,0.751,0.246,0.001,0.001,0.001,0,Live Till We Die,Elijah Blake
2,0.664,0.131,0.002,0.002,0.202,0,The Otherside,Elijah Blake
3,0.199,0.690,0.001,0.001,0.109,1,Pinot,Elijah Blake
4,0.125,0.322,0.168,0.003,0.382,4,Shadows & Diamonds,Elijah Blake
...,...,...,...,...,...,...,...,...
158675,0.001,0.909,0.001,0.001,0.088,1,In The Party,Flo Milli
158676,0.001,0.805,0.001,0.001,0.192,1,Beef FloMix,Flo Milli
158677,0.001,0.696,0.001,0.038,0.265,1,Rodeo (Remix),Flo Milli
158678,0.001,0.996,0.001,0.001,0.001,1,Conceited,Flo Milli


In [71]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,0,66713
1,4,36323
2,1,20714
3,3,18107
4,2,16823


In [72]:
df_document_topic.to_csv('../lda-results/document_topic_with_lyrics_and_top_10_no_stop_words.csv')
df_topic_distribution.to_csv('../lda-results/topic_distribution_with_lyrics_and_top_10_no_stop_words.csv')

In [37]:
from sklearn.model_selection import GridSearchCV

# We are going to test multiple values for the number of topics
search_params = {'n_components': [3, 4, 5]}

# Initialize the LDA model
lda = LatentDirichletAllocation()

# Initialize a Grid Search with cross-validation instance
grid = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
grid.fit(dtm)

GridSearchCV(estimator=LatentDirichletAllocation(),
             param_grid={'n_components': [3, 4, 5]})

In [38]:
grid.cv_results_

{'mean_fit_time': array([322.20832949, 342.09373679, 344.99867043]),
 'std_fit_time': array([22.70609384, 15.30608217,  4.5620314 ]),
 'mean_score_time': array([5.43091664, 6.09414873, 6.06560345]),
 'std_score_time': array([0.43705359, 0.36575259, 0.13991072]),
 'param_n_components': masked_array(data=[3, 4, 5],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_components': 3}, {'n_components': 4}, {'n_components': 5}],
 'split0_test_score': array([-23046081.20111998, -23050772.35207721, -23012318.59184919]),
 'split1_test_score': array([-23663310.51873978, -23555950.55534031, -23633661.00556118]),
 'split2_test_score': array([-23765093.48499016, -23865413.89566913, -23782554.40758479]),
 'split3_test_score': array([-24209871.31897045, -24168308.25315203, -24168346.24125382]),
 'split4_test_score': array([-23689125.6136878 , -23637105.46491072, -23640879.92009572]),
 'mean_test_score': array([-23674696.42750163, -23655510.104

In [39]:
# Best Model
best_lda_model = grid.best_estimator_

# Model Parameters
print("Best Model's Params: ", grid.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", grid.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(dtm))

Best Model's Params:  {'n_components': 5}
Best Log Likelihood Score:  -23647552.03326894
Model Perplexity:  1956.6398390262623
